In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation,Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization,Reshape,InputLayer,GlobalAveragePooling2D,DepthwiseConv2D,Dropout,MaxPooling2D,ZeroPadding2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

import itertools

In [2]:
train_path='train'
test_path='test'
valid_path='valid'

In [6]:
import os
os.chdir("Air Dataset")

In [7]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path,target_size=(512,512),batch_size=32)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path,target_size=(512,512),batch_size=4)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path,target_size=(512,512),batch_size=4,shuffle=False)


Found 2244 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [24]:
mobile=tf.keras.applications.mobilenet.MobileNet()


MobileNet v1Lite without Bitshift

In [8]:
model=Sequential()
model.add(InputLayer(shape=(512, 512, 3)))

In [27]:
for layer in mobile.layers[1:43]:
     if isinstance(layer, Conv2D):
        
        #print(f'Layer {layer.name} is a Conv2D layer with {layer.filters} filters')
        newfilters=(int)(layer.filters/4)
        newlayer=Conv2D(newfilters, layer.kernel_size, strides=layer.strides,
                      padding=layer.padding, activation=layer.activation,
                      use_bias=layer.use_bias, kernel_initializer=layer.kernel_initializer,
                      bias_initializer=layer.bias_initializer)
        model.add(newlayer)
        print(f'Layer {newlayer.name} is a Conv2D layer with {newlayer.filters} filters and {newlayer.strides} strides and {newlayer.kernel_size} kernel_size')
     elif isinstance(layer, BatchNormalization) :
        model.add(BatchNormalization(axis=getattr(layer, 'axis', -1),
    momentum=getattr(layer, 'momentum', 0.99),
    epsilon=getattr(layer, 'epsilon', 1e-3),
    center=getattr(layer, 'center', True),
    scale=getattr(layer, 'scale', True),
    beta_initializer=getattr(layer, 'beta_initializer', 'zeros'),
    gamma_initializer=getattr(layer, 'gamma_initializer', 'ones'),
    moving_mean_initializer=getattr(layer, 'moving_mean_initializer', 'zeros'),
    moving_variance_initializer=getattr(layer, 'moving_variance_initializer', 'ones'),
    beta_regularizer=getattr(layer, 'beta_regularizer', None),
    gamma_regularizer=getattr(layer, 'gamma_regularizer', None),
    beta_constraint=getattr(layer, 'beta_constraint', None),
    gamma_constraint=getattr(layer, 'gamma_constraint', None)))
     elif isinstance(layer,DepthwiseConv2D):
         print(f'Layer {layer.name} is a DepthConv2D layer with  {layer.strides} strides and {layer.kernel_size} kernel size')
         model.add(DepthwiseConv2D(kernel_size=getattr(layer, 'kernel_size', (3, 3)),
    strides=getattr(layer, 'strides', (1, 1)),
    padding=getattr(layer, 'padding', 'valid'),
    depth_multiplier=getattr(layer, 'depth_multiplier', 1),
    activation=getattr(layer, 'activation', None),
    use_bias=getattr(layer, 'use_bias', True),
    bias_initializer=getattr(layer, 'bias_initializer', 'zeros'),
    depthwise_initializer=getattr(layer, 'depthwise_initializer', 'glorot_uniform')
))
     #elif(isinstance(layer,DepthwiseConv2D)):
        #     print('hellooooooooooo')
       #      print(f'Layer {layer.name} is a DepthConv2D layer with  {layer.strides} strides')
             #model.add(layer)
     elif (isinstance(layer,ZeroPadding2D)):
         print(1)

     else:
          #print(layer.name)
          model.add(layer)
     
   

Layer conv2d_7 is a Conv2D layer with 8 filters and (2, 2) strides and (3, 3) kernel_size
Layer conv_dw_1 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_8 is a Conv2D layer with 16 filters and (1, 1) strides and (1, 1) kernel_size
1
Layer conv_dw_2 is a DepthConv2D layer with  (2, 2) strides and (3, 3) kernel size
Layer conv2d_9 is a Conv2D layer with 32 filters and (1, 1) strides and (1, 1) kernel_size
Layer conv_dw_3 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_10 is a Conv2D layer with 32 filters and (1, 1) strides and (1, 1) kernel_size
1
Layer conv_dw_4 is a DepthConv2D layer with  (2, 2) strides and (3, 3) kernel size
Layer conv2d_11 is a Conv2D layer with 64 filters and (1, 1) strides and (1, 1) kernel_size
Layer conv_dw_5 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_12 is a Conv2D layer with 64 filters and (1, 1) strides and (1, 1) kernel_size
1
Layer conv_dw_6 is a DepthConv2

In [28]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 8)    │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 256, 256, 8)    │            72 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 16)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 256, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 127, 127, 16)   │           144 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 127, 127, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 7, 7, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 127, 127, 32)   │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_2              │ (None, 127, 127, 32)   │           288 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 127, 127, 32)   │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │             

 Total params: 49,800 (194.53 KB)

 Trainable params: 47,560 (185.78 KB)

 Non-trainable params: 2,240 (8.75 KB)

In [29]:
for layer in mobile.layers[73:86]:
     if isinstance(layer, Conv2D):
        
        #print(f'Layer {layer.name} is a Conv2D layer with {layer.filters} filters')
        newfilters=(int)(layer.filters/4)
        newlayer=Conv2D(newfilters, layer.kernel_size, strides=layer.strides,
                      padding=layer.padding, activation=layer.activation,
                      use_bias=layer.use_bias, kernel_initializer=layer.kernel_initializer,
                      bias_initializer=layer.bias_initializer)
        model.add(newlayer)
        print(f'Layer {newlayer.name} is a Conv2D layer with {newlayer.filters} filters and {newlayer.strides} strides and {newlayer.kernel_size} kernel_size')
     elif isinstance(layer, BatchNormalization) :
        model.add(BatchNormalization(axis=getattr(layer, 'axis', -1),
    momentum=getattr(layer, 'momentum', 0.99),
    epsilon=getattr(layer, 'epsilon', 1e-3),
    center=getattr(layer, 'center', True),
    scale=getattr(layer, 'scale', True),
    beta_initializer=getattr(layer, 'beta_initializer', 'zeros'),
    gamma_initializer=getattr(layer, 'gamma_initializer', 'ones'),
    moving_mean_initializer=getattr(layer, 'moving_mean_initializer', 'zeros'),
    moving_variance_initializer=getattr(layer, 'moving_variance_initializer', 'ones'),
    beta_regularizer=getattr(layer, 'beta_regularizer', None),
    gamma_regularizer=getattr(layer, 'gamma_regularizer', None),
    beta_constraint=getattr(layer, 'beta_constraint', None),
    gamma_constraint=getattr(layer, 'gamma_constraint', None)))
     elif isinstance(layer,DepthwiseConv2D):
         print(f'Layer {layer.name} is a DepthConv2D layer with  {layer.strides} strides and {layer.kernel_size} kernel size')

         model.add(DepthwiseConv2D(kernel_size=getattr(layer, 'kernel_size', (3, 3)),
    strides=getattr(layer, 'strides', (1, 1)),
    padding=getattr(layer, 'padding', 'valid'),
    depth_multiplier=getattr(layer, 'depth_multiplier', 1),
    activation=getattr(layer, 'activation', None),
    use_bias=getattr(layer, 'use_bias', True),
    bias_initializer=getattr(layer, 'bias_initializer', 'zeros'),
    depthwise_initializer=getattr(layer, 'depthwise_initializer', 'glorot_uniform')
))
     elif (isinstance(layer,ZeroPadding2D)):
         print(1) 
     else:
         model.add(layer)

   

1
Layer conv_dw_12 is a DepthConv2D layer with  (2, 2) strides and (3, 3) kernel size
Layer conv2d_14 is a Conv2D layer with 256 filters and (1, 1) strides and (1, 1) kernel_size
Layer conv_dw_13 is a DepthConv2D layer with  (1, 1) strides and (3, 3) kernel size
Layer conv2d_15 is a Conv2D layer with 256 filters and (1, 1) strides and (1, 1) kernel_size


In [30]:
model.add(GlobalAveragePooling2D(name='global_average_pooling2d'))
model.add(Dense(units=4,activation='softmax'))

In [35]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 8)    │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 256, 256, 8)    │            72 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 16)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 256, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 127, 127, 16)   │           144 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 127, 127, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 7, 7, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 127, 127, 32)   │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_2              │ (None, 127, 127, 32)   │           288 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 127, 127, 32)   │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │             

 Total params: 156,172 (610.05 KB)

 Trainable params: 152,140 (594.30 KB)

 Non-trainable params: 4,032 (15.75 KB)

In [32]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])


In [34]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=20,verbose=2)


Epoch 1/20



KeyboardInterrupt



In [13]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=20,verbose=2)


Epoch 1/20


c:\Users\HP\Documents\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


71/71 - 253s - 4s/step - accuracy: 0.6872 - loss: 0.7596 - val_accuracy: 0.2500 - val_loss: 1.5832
Epoch 2/20
71/71 - 221s - 3s/step - accuracy: 0.7736 - loss: 0.5615 - val_accuracy: 0.2500 - val_loss: 2.1149
Epoch 3/20
71/71 - 223s - 3s/step - accuracy: 0.7981 - loss: 0.4750 - val_accuracy: 0.2500 - val_loss: 2.7135
Epoch 4/20
71/71 - 225s - 3s/step - accuracy: 0.8191 - loss: 0.4346 - val_accuracy: 0.2500 - val_loss: 3.4659
Epoch 5/20
71/71 - 219s - 3s/step - accuracy: 0.8258 - loss: 0.4037 - val_accuracy: 0.2500 - val_loss: 3.5441
Epoch 6/20
71/71 - 224s - 3s/step - accuracy: 0.8373 - loss: 0.3888 - val_accuracy: 0.2500 - val_loss: 3.6216
Epoch 7/20
71/71 - 220s - 3s/step - accuracy: 0.8454 - loss: 0.3495 - val_accuracy: 0.2917 - val_loss: 3.0772
Epoch 8/20
71/71 - 229s - 3s/step - accuracy: 0.8614 - loss: 0.3237 - val_accuracy: 0.6083 - val_loss: 0.9164
Epoch 9/20
71/71 - 207s - 3s/step - accuracy: 0.8739 - loss: 0.2971 - val_accuracy: 0.7417 - val_loss: 0.6955
Epoch 10/20
71/71 - 2

In [14]:
predictions=model.predict(x=test_batches,verbose=0)
rounded_pred=np.round(predictions)

In [15]:
predict=np.argmax(rounded_pred,axis=1)


In [16]:
from sklearn.metrics import f1_score,accuracy_score
test_labels=test_batches.classes

f1_micro = f1_score(test_labels, predict,average='micro')
f1_micro*100

91.66666666666666

In [18]:
model.export('mobilenetv1lite')

INFO:tensorflow:Assets written to: mobilenetv1lite\assets


INFO:tensorflow:Assets written to: mobilenetv1lite\assets


Saved artifact at 'mobilenetv1lite'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name='keras_tensor_91')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2609415171024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415172368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415171216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415172176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415172560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415174288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415174096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415175056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415174864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415173712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2609415176016: TensorSpec(shape=

In [23]:
model.save_weights('weights.weights.h5')


In [25]:
model.save('mobilenetv1lite.keras')


MobileNet v1 Lite with bitshift

In [9]:
from keras.models import load_model
mobilenetv1lite=load_model('mobilenetv1lite.keras')

In [10]:
mobilenetv1liteQuantized=Sequential()

In [11]:
for layer in mobilenetv1lite.layers[:-1]:
    mobilenetv1liteQuantized.add(layer)

In [12]:
mobilenetv1liteQuantized.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 8)    │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 256, 256, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 256, 256, 8)    │            72 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 256, 256, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 16)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 256, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 256, 256, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 127, 127, 16)   │           144 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 127, 127, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 127, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 127, 127, 32)   │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_2              │ (None, 127, 127, 32)   │           288 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 127, 127, 32)   │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 127, 127, 32)   │           128 │
│ (BatchNormalization)            │                        │             

 Total params: 125,744 (491.19 KB)

 Trainable params: 122,832 (479.81 KB)

 Non-trainable params: 2,912 (11.38 KB)

In [13]:
os.chdir('..')
os.getcwd()

'c:\\Users\\HP\\Documents\\GitHub\\CNN-ON-FPGA-MSc.-\\Training Model(Python)'

In [14]:
import ml_dtypes

from keras.src import activations
from keras.src import constraints
from keras.src import dtype_policies
from keras.src import initializers
from keras.src import ops
from keras.src import quantizers
from keras.src import regularizers
from keras.src.api_export import keras_export
from keras.src.layers.input_spec import InputSpec
from keras.src.layers.layer import Layer
import numpy 
@tf.custom_gradient
def MatrixBitShift(A,B,D):
    def grd(dy):
        G=ops.matmult(dy,ops.transpose(D))
        return G

    A=tf.cast(A,tf.int32)
    B=tf.cast(B,tf.int32)
    columns2=tf.shape(B)[1]
    rows2=tf.shape(B)[0]
    rows1=tf.shape(A)[0]
    C=tf.zeros(shape=(rows1,columns2),dtype=tf.int32)
    for i in range(0,rows1):
      for j in range(0,rows2):
            for  k in range(0,columns2):
                original=C[i][k]
                a=A[i][j]
                b=B[j][k]
                x=bitshift(a,b)
                #print(x)
                update=original+x
                
                index=tf.constant([[i,k]])
                #u=tf.constant([update],dtype=tf.int32)
                C=tf.tensor_scatter_nd_update(C,index,[update])
    return C,grd

def Quantizeweights(weights):
    #weights=weights.numpy()
    weights_absolute=tf.abs(weights)
    maxweight=tf.reduce_max(weights_absolute)
    #print(maxweight)
    scaled_weights=weights/maxweight
    #print(scaled_weights)
    lq=LQ(scaled_weights,4)
    #print(f'aloha {lq}')
    return lq
@tf.custom_gradient
def bitshift(weight,shift):
  res=0
  if shift<0:
       y=tf.math.abs(shift)
       res= tf.bitwise.right_shift(weight,y)
  else:
       res= tf.bitwise.left_shift(weight,shift)
  def grad(dy):
       dw=2**shift*dy
       ds=2**shift*weight*tf.math.log(2)*dy
       return dw,ds
  return res,grad
    
    #pot_vect=numpy.vectorize(powertwo)
    #final_weights=pot_vect(lq,maxweight)
    #print(final_weights)



def check_integer_zero_and_first_two_decimals_zero(number):
    # Check if the integer part is zero
    if int(number) != 0:
        return False
    
    # Convert the number to a string
    num_str = str(number)
    
    # Find the decimal point
    decimal_point_index = num_str.find('.')
    
    # Check if there is a decimal point and if the length after the decimal point is at least 2
    if decimal_point_index != -1 and len(num_str) > decimal_point_index + 2:
        # Get the first two decimal places
        first_two_decimals = num_str[decimal_point_index + 1:decimal_point_index + 3]
        # Check if the first two decimal places are "00"
        return first_two_decimals == "00"
    
    # If there are no decimal places or less than two decimal places, return False
    return False
def clip(weight,bitwidth):
    
    weight_log=numpy.round(numpy.log2(numpy.absolute(weight)))
    bit_pot=-2**bitwidth
    if weight_log<bit_pot:
        return 0
    elif weight_log>=0:
        return -1
    else:
        return weight_log
def LQ(tensor, bitwidth):
    # Create a mask for weights equal to 0
    #rounded_tensor=tf.where(tensor>0,tf.my u4xdcvp]
    # [6ath.floor(tensor),tf.math.ceil(tensor))
    rounded_weights=tensor*100
    rounded_weights=tf.where(rounded_weights>0,tf.math.floor(rounded_weights),tf.math.ceil(rounded_weights))
    epsilon = 1e-7

    # Compute weight_log using log2
    weight_log = tf.math.log(tf.math.abs(tensor)+epsilon) / tf.math.log(2.0)
    #print(weight_log)
    weight_log=tf.where(weight_log<0,tf.math.ceil(weight_log),tf.math.floor(weight_log))
    bit_pot = -2 ** bitwidth
    mask_condition = weight_log >= bit_pot
    
    # Initialize the result tensor with zeros
    result = tf.zeros_like(tensor, dtype=tf.float32)
    
    # Apply the condition where weight_log is valid
    result = tf.where(rounded_weights!=0,tf.where(mask_condition, tf.where(tensor > 0, tf.math.abs(weight_log), weight_log), result),result)
    
    return result

def powertwo(weight,sf):
    pot_weight=(2**weight)*sf
    return pot_weight
    

class QDense(Layer):
    """Just your regular densely-connected NN layer with Quantized weights and bit shifting.

    `Dense` implements the operation:
    `output = activation(dot(input, kernel) + bias)`
    where `activation` is the element-wise activation function
    passed as the `activation` argument, `kernel` is a weights matrix
    created by the layer, and `bias` is a bias vector created by the layer
    (only applicable if `use_bias` is `True`).

    Note: If the input to the layer has a rank greater than 2, `Dense`
    computes the dot product between the `inputs` and the `kernel` along the
    last axis of the `inputs` and axis 0 of the `kernel` (using `tf.tensordot`).
    For example, if input has dimensions `(batch_size, d0, d1)`, then we create
    a `kernel` with shape `(d1, units)`, and the `kernel` operates along axis 2
    of the `input`, on every sub-tensor of shape `(1, 1, d1)` (there are
    `batch_size * d0` such sub-tensors). The output in this case will have
    shape `(batch_size, d0, units)`.

    Args:
        units: Positive integer, dimensionality of the output space.
        activation: Activation function to use.
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix.
        bias_initializer: Initializer for the bias vector.
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix.
        bias_regularizer: Regularizer function applied to the bias vector.
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix.
        bias_constraint: Constraint function applied to the bias vector.
        lora_rank: Optional integer. If set, the layer's forward pass
            will implement LoRA (Low-Rank Adaptation)
            with the provided rank. LoRA sets the layer's kernel
            to non-trainable and replaces it with a delta over the
            original kernel, obtained via multiplying two lower-rank
            trainable matrices. This can be useful to reduce the
            computation cost of fine-tuning large dense layers.
            You can also enable LoRA on an existing
            `Dense` layer by calling `layer.enable_lora(rank)`.

    Input shape:
        N-D tensor with shape: `(batch_size, ..., input_dim)`.
        The most common situation would be
        a 2D input with shape `(batch_size, input_dim)`.

    Output shape:
        N-D tensor with shape: `(batch_size, ..., units)`.
        For instance, for a 2D input with shape `(batch_size, input_dim)`,
        the output would have shape `(batch_size, units)`.
    """

    def __init__(
        self,
        units,
        activation=None,
        use_bias=True,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        lora_rank=None,
        **kwargs,
    ):
        super().__init__(activity_regularizer=activity_regularizer, **kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.lora_rank = lora_rank
        self.lora_enabled = False
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        input_dim = input_shape[-1]
        # We use `self._dtype_policy` to check to avoid issues in torch dynamo
        is_quantized = isinstance(
            self._dtype_policy, dtype_policies.QuantizedDTypePolicy
        )
        if is_quantized:
            self.quantized_build(
                input_shape, mode=self.dtype_policy.quantization_mode
            )
        if not is_quantized or self.dtype_policy.quantization_mode != "int8":
            # If the layer is quantized to int8, `self._kernel` will be added
            # in `self._int8_build`. Therefore, we skip it here.
            self._kernel = self.add_weight(
                name="kernel",
                shape=(input_dim, self.units),
                initializer=self.kernel_initializer,
                regularizer=self.kernel_regularizer,
                constraint=self.kernel_constraint,
            )
        if self.use_bias:
            self.bias = self.add_weight(
                name="bias",
                shape=(self.units,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
            )
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True
        if self.lora_rank:
            self.enable_lora(self.lora_rank)

    @property
    def kernel(self):
        if not self.built:
            raise AttributeError(
                "You must build the layer before accessing `kernel`."
            )
        if self.lora_enabled:
            return self._kernel + ops.matmul(
                self.lora_kernel_a, self.lora_kernel_b
            )
        return self._kernel

    def call(self, inputs):
        
        quantized_weights=Quantizeweights(self.kernel)
        #x = ops.matmul(inputs, self.kernel)
        x=MatrixBitShift(inputs,quantized_weights,self.kernel)
        if self.bias is not None:
            x = ops.add(x, self.bias)
        if self.activation is not None:
            x = self.activation(x)
        return x

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def enable_lora(
        self, rank, a_initializer="he_uniform", b_initializer="zeros"
    ):
        if self.kernel_constraint:
            raise ValueError(
                "Lora is incompatible with kernel constraints. "
                "In order to enable lora on this layer, remove the "
                "`kernel_constraint` argument."
            )
        if not self.built:
            raise ValueError(
                "Cannot enable lora on a layer that isn't yet built."
            )
        if self.lora_enabled:
            raise ValueError(
                "lora is already enabled. "
                "This can only be done once per layer."
            )
        self._tracker.unlock()
        self.lora_kernel_a = self.add_weight(
            name="lora_kernel_a",
            shape=(self.kernel.shape[0], rank),
            initializer=initializers.get(a_initializer),
            regularizer=self.kernel_regularizer,
        )
        self.lora_kernel_b = self.add_weight(
            name="lora_kernel_b",
            shape=(rank, self.kernel.shape[1]),
            initializer=initializers.get(b_initializer),
            regularizer=self.kernel_regularizer,
        )
        self._kernel.trainable = False
        self._tracker.lock()
        self.lora_enabled = True
        self.lora_rank = rank

    def save_own_variables(self, store):
        # Do nothing if the layer isn't yet built
        if not self.built:
            return
        # The keys of the `store` will be saved as determined because the
        # default ordering will change after quantization
        kernel_value, kernel_scale = self._get_kernel_with_merged_lora()
        target_variables = [kernel_value]
        if self.use_bias:
            target_variables.append(self.bias)
        if isinstance(self.dtype_policy, dtype_policies.QuantizedDTypePolicy):
            mode = self.dtype_policy.quantization_mode
            if mode == "int8":
                target_variables.append(kernel_scale)
            elif mode == "float8":
                target_variables.append(self.inputs_scale)
                target_variables.append(self.inputs_amax_history)
                target_variables.append(self.kernel_scale)
                target_variables.append(self.kernel_amax_history)
                target_variables.append(self.outputs_grad_scale)
                target_variables.append(self.outputs_grad_amax_history)
            else:
                raise NotImplementedError(
                    self.QUANTIZATION_MODE_ERROR_TEMPLATE.format(mode)
                )
        for i, variable in enumerate(target_variables):
            store[str(i)] = variable

    def load_own_variables(self, store):
        if not self.lora_enabled:
            self._check_load_own_variables(store)
        # Do nothing if the layer isn't yet built
        if not self.built:
            return
        # The keys of the `store` will be saved as determined because the
        # default ordering will change after quantization
        target_variables = [self._kernel]
        if self.use_bias:
            target_variables.append(self.bias)
        if isinstance(self.dtype_policy, dtype_policies.QuantizedDTypePolicy):
            mode = self.dtype_policy.quantization_mode
            if mode == "int8":
                target_variables.append(self.kernel_scale)
            elif mode == "float8":
                target_variables.append(self.inputs_scale)
                target_variables.append(self.inputs_amax_history)
                target_variables.append(self.kernel_scale)
                target_variables.append(self.kernel_amax_history)
                target_variables.append(self.outputs_grad_scale)
                target_variables.append(self.outputs_grad_amax_history)
            else:
                raise NotImplementedError(
                    self.QUANTIZATION_MODE_ERROR_TEMPLATE.format(mode)
                )
        for i, variable in enumerate(target_variables):
            variable.assign(store[str(i)])
        if self.lora_enabled:
            self.lora_kernel_a.assign(ops.zeros(self.lora_kernel_a.shape))
            self.lora_kernel_b.assign(ops.zeros(self.lora_kernel_b.shape))

    def get_config(self):
        base_config = super().get_config()
        config = {
            "units": self.units,
            "activation": activations.serialize(self.activation),
            "use_bias": self.use_bias,
            "kernel_initializer": initializers.serialize(
                self.kernel_initializer
            ),
            "bias_initializer": initializers.serialize(self.bias_initializer),
            "kernel_regularizer": regularizers.serialize(
                self.kernel_regularizer
            ),
            "bias_regularizer": regularizers.serialize(self.bias_regularizer),
            "kernel_constraint": constraints.serialize(self.kernel_constraint),
            "bias_constraint": constraints.serialize(self.bias_constraint),
        }
        if self.lora_rank:
            config["lora_rank"] = self.lora_rank
        return {**base_config, **config}

    def _check_load_own_variables(self, store):
        all_vars = self._trainable_variables + self._non_trainable_variables
        if len(store.keys()) != len(all_vars):
            if len(all_vars) == 0 and not self.built:
                raise ValueError(
                    f"Layer '{self.name}' was never built "
                    "and thus it doesn't have any variables. "
                    f"However the weights file lists {len(store.keys())} "
                    "variables for this layer.\n"
                    "In most cases, this error indicates that either:\n\n"
                    "1. The layer is owned by a parent layer that "
                    "implements a `build()` method, but calling the "
                    "parent's `build()` method did NOT create the state of "
                    f"the child layer '{self.name}'. A `build()` method "
                    "must create ALL state for the layer, including "
                    "the state of any children layers.\n\n"
                    "2. You need to implement "
                    "the `def build_from_config(self, config)` method "
                    f"on layer '{self.name}', to specify how to rebuild "
                    "it during loading. "
                    "In this case, you might also want to implement the "
                    "method that generates the build config at saving time, "
                    "`def get_build_config(self)`. "
                    "The method `build_from_config()` is meant "
                    "to create the state "
                    "of the layer (i.e. its variables) upon deserialization.",
                )
            raise ValueError(
                f"Layer '{self.name}' expected {len(all_vars)} variables, "
                "but received "
                f"{len(store.keys())} variables during loading. "
                f"Expected: {[v.name for v in all_vars]}"
            )

    """Quantization-related (int8 and float8) methods"""

    QUANTIZATION_MODE_ERROR_TEMPLATE = (
        f"Invalid quantization mode. Expected one of "
        f"{dtype_policies.QUANTIZATION_MODES}. "
        "Received: quantization_mode={mode}"
    )

    def quantized_build(self, input_shape, mode):
        if mode == "int8":
            input_dim = input_shape[-1]
            kernel_shape = (input_dim, self.units)
            self._int8_build(kernel_shape)
        elif mode == "float8":
            self._float8_build()
        else:
            raise NotImplementedError(
                self.QUANTIZATION_MODE_ERROR_TEMPLATE.format(mode)
            )

    def _int8_build(
        self,
        kernel_shape,
        kernel_initializer="zeros",
        kernel_scale_initializer="ones",
    ):
        self.inputs_quantizer = quantizers.AbsMaxQuantizer(axis=-1)
        self._kernel = self.add_weight(
            name="kernel",
            shape=kernel_shape,
            initializer=kernel_initializer,
            dtype="int8",
            trainable=False,
        )
        self.kernel_scale = self.add_weight(
            name="kernel_scale",
            shape=(self.units,),
            initializer=kernel_scale_initializer,
            trainable=False,
        )
        self._is_quantized = True

    def _float8_build(self):
        if not isinstance(
            self.dtype_policy, dtype_policies.QuantizedFloat8DTypePolicy
        ):
            raise TypeError(
                "`self.dtype_policy` must be the type of "
                f"QuantizedFloat8DTypePolicy. Received {self.dtype_policy}"
            )
        amax_history_length = self.dtype_policy.amax_history_length
        # We set `trainable=True` because we will use the gradients to overwrite
        # these variables
        scale_kwargs = {
            "shape": (),
            "initializer": "ones",
            "dtype": "float32",  # Always be float32
            "trainable": True,
            "autocast": False,
        }
        amax_history_kwargs = {
            "shape": (amax_history_length,),
            "initializer": "zeros",
            "dtype": "float32",  # Always be float32
            "trainable": True,
            "autocast": False,
        }
        self.inputs_scale = self.add_weight(name="inputs_scale", **scale_kwargs)
        self.inputs_amax_history = self.add_weight(
            name="inputs_amax_history", **amax_history_kwargs
        )
        self.kernel_scale = self.add_weight(name="kernel_scale", **scale_kwargs)
        self.kernel_amax_history = self.add_weight(
            name="kernel_amax_history", **amax_history_kwargs
        )
        self.outputs_grad_scale = self.add_weight(
            name="outputs_grad_scale", **scale_kwargs
        )
        self.outputs_grad_amax_history = self.add_weight(
            name="outputs_grad_amax_history", **amax_history_kwargs
        )
        # We need to set `overwrite_with_gradient=True` to instruct the
        # optimizer to directly overwrite these variables with their computed
        # gradients during training
        self.inputs_scale.overwrite_with_gradient = True
        self.inputs_amax_history.overwrite_with_gradient = True
        self.kernel_scale.overwrite_with_gradient = True
        self.kernel_amax_history.overwrite_with_gradient = True
        self.outputs_grad_scale.overwrite_with_gradient = True
        self.outputs_grad_amax_history.overwrite_with_gradient = True
        self._is_quantized = True

    def quantized_call(self, inputs):
        if self.dtype_policy.quantization_mode == "int8":
            return self._int8_call(inputs)
        elif self.dtype_policy.quantization_mode == "float8":
            return self._float8_call(inputs)
        else:
            mode = self.dtype_policy.quantization_mode
            raise NotImplementedError(
                self.QUANTIZATION_MODE_ERROR_TEMPLATE.format(mode)
            )

    def _int8_call(self, inputs):
        @ops.custom_gradient
        def matmul_with_inputs_gradient(inputs, kernel, kernel_scale):
            def grad_fn(*args, upstream=None):
                if upstream is None:
                    (upstream,) = args
                float_kernel = ops.divide(
                    ops.cast(kernel, dtype=self.compute_dtype),
                    kernel_scale,
                )
                inputs_grad = ops.matmul(upstream, ops.transpose(float_kernel))
                return (inputs_grad, None, None)

            inputs, inputs_scale = self.inputs_quantizer(inputs)
            x = ops.matmul(inputs, kernel)
            # De-scale outputs
            x = ops.cast(x, self.compute_dtype)
            x = ops.divide(x, ops.multiply(inputs_scale, kernel_scale))
            return x, grad_fn

        x = matmul_with_inputs_gradient(
            inputs,
            ops.convert_to_tensor(self._kernel),
            ops.convert_to_tensor(self.kernel_scale),
        )
        if self.lora_enabled:
            lora_x = ops.matmul(inputs, self.lora_kernel_a)
            lora_x = ops.matmul(lora_x, self.lora_kernel_b)
            x = ops.add(x, lora_x)
        if self.bias is not None:
            x = ops.add(x, self.bias)
        if self.activation is not None:
            x = self.activation(x)
        return x

    def _float8_call(self, inputs):
        if self.lora_enabled:
            raise NotImplementedError(
                "Currently, `_float8_call` doesn't support LoRA"
            )

        @ops.custom_gradient
        def quantized_dequantize_inputs(inputs, scale, amax_history):
            new_scale = quantizers.compute_float8_scale(
                ops.max(amax_history, axis=0),
                scale,
                ops.cast(
                    float(ml_dtypes.finfo("float8_e4m3fn").max), "float32"
                ),
            )
            qdq_inputs = quantizers.quantize_and_dequantize(
                inputs, scale, "float8_e4m3fn", self.compute_dtype
            )
            new_amax_history = quantizers.compute_float8_amax_history(
                inputs, amax_history
            )

            def grad(*args, upstream=None, variables=None):
                if upstream is None:
                    (upstream,) = args
                return upstream, new_scale, new_amax_history

            return qdq_inputs, grad

        @ops.custom_gradient
        def quantized_dequantize_outputs(outputs, scale, amax_history):
            """Quantize-dequantize the output gradient but not the output."""

            def grad(*args, upstream=None, variables=None):
                if upstream is None:
                    (upstream,) = args
                new_scale = quantizers.compute_float8_scale(
                    ops.max(amax_history, axis=0),
                    scale,
                    ops.cast(
                        float(ml_dtypes.finfo("float8_e5m2").max), "float32"
                    ),
                )
                qdq_upstream = quantizers.quantize_and_dequantize(
                    upstream, scale, "float8_e5m2", self.compute_dtype
                )
                new_amax_history = quantizers.compute_float8_amax_history(
                    upstream, amax_history
                )
                return qdq_upstream, new_scale, new_amax_history

            return outputs, grad

        x = ops.matmul(
            quantized_dequantize_inputs(
                inputs,
                ops.convert_to_tensor(self.inputs_scale),
                ops.convert_to_tensor(self.inputs_amax_history),
            ),
            quantized_dequantize_inputs(
                ops.convert_to_tensor(self._kernel),
                ops.convert_to_tensor(self.kernel_scale),
                ops.convert_to_tensor(self.kernel_amax_history),
            ),
        )
        # `quantized_dequantize_outputs` is placed immediately after
        # `ops.matmul` for the sake of pattern matching in gemm_rewrite. That
        # way, the qdq will be adjacent to the corresponding matmul_bprop in the
        # bprop.
        x = quantized_dequantize_outputs(
            x,
            ops.convert_to_tensor(self.outputs_grad_scale),
            ops.convert_to_tensor(self.outputs_grad_amax_history),
        )
        if self.bias is not None:
            # Under non-mixed precision cases, F32 bias has to be converted to
            # BF16 first to get the biasAdd fusion support. ref. PR
            # https://github.com/tensorflow/tensorflow/pull/60306
            bias = self.bias
            if self.dtype_policy.compute_dtype == "float32":
                bias_bf16 = ops.cast(bias, "bfloat16")
                bias = ops.cast(bias_bf16, bias.dtype)
            x = ops.add(x, bias)
        if self.activation is not None:
            x = self.activation(x)
        return x

    def quantize(self, mode):
        import gc

        # Prevent quantization of the subclasses
        if type(self) is not Dense:
            raise NotImplementedError(
                f"Layer {self.__class__.__name__} does not have a `quantize()` "
                "method implemented."
            )
        self._check_quantize_args(mode, self.compute_dtype)

        # Set new dtype policy
        if not isinstance(
            self.dtype_policy, dtype_policies.QuantizedDTypePolicy
        ):
            quantized_dtype = f"{mode}_from_{self.dtype_policy.name}"
            # We set the internal `self._dtype_policy` instead of using the
            # setter to avoid double `quantize` call
            self._dtype_policy = dtype_policies.get(quantized_dtype)

        self._tracker.unlock()
        if mode == "int8":
            # Quantize `self._kernel` to int8 and compute corresponding scale
            kernel_value, kernel_scale = quantizers.abs_max_quantize(
                self._kernel, axis=0
            )
            kernel_scale = ops.squeeze(kernel_scale, axis=0)
            self._untrack_variable(self._kernel)
            kernel_shape = self._kernel.shape
            del self._kernel
            # Utilize a lambda expression as an initializer to prevent adding a
            # large constant to the computation graph.
            self._int8_build(
                kernel_shape,
                lambda shape, dtype: kernel_value,
                lambda shape, dtype: kernel_scale,
            )
        elif mode == "float8":
            self._float8_build()
        else:
            raise NotImplementedError(
                self.QUANTIZATION_MODE_ERROR_TEMPLATE.format(mode)
            )
        self._tracker.lock()

        # Release memory manually because sometimes the backend doesn't
        gc.collect()

    def _get_kernel_with_merged_lora(self):
        if isinstance(self.dtype_policy, dtype_policies.QuantizedDTypePolicy):
            kernel_value = self._kernel
            kernel_scale = self.kernel_scale
            if self.lora_enabled:
                # Dequantize & quantize to merge lora weights into int8 kernel
                # Note that this is a lossy compression
                kernel_value = ops.divide(kernel_value, kernel_scale)
                kernel_value = ops.add(
                    kernel_value,
                    ops.matmul(self.lora_kernel_a, self.lora_kernel_b),
                )
                kernel_value, kernel_scale = quantizers.abs_max_quantize(
                    kernel_value, axis=0
                )
                kernel_scale = ops.squeeze(kernel_scale, axis=0)
            return kernel_value, kernel_scale
        return self.kernel, None
    

In [78]:
mobilenetv1liteQuantized.add(QDense(units=4,activation='softmax'))


In [79]:
mobilenetv1liteQuantized.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (32, 256, 256, 8)      │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (32, 256, 256, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (32, 256, 256, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (32, 256, 256, 8)      │            72 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (32, 256, 256, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (32, 256, 256, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (32, 256, 256, 16)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (32, 256, 256, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (32, 256, 256, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (32, 127, 127, 16)     │           144 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (32, 127, 127, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (32, 127, 127, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (32, 127, 127, 32)     │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (32, 127, 127, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_2              │ (32, 127, 127, 32)     │           288 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (32, 127, 127, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (32, 127, 127, 32)     │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (32, 127, 127, 32)     │           128 │
│ (BatchNormalization)            │                        │             

 Total params: 125,744 (491.19 KB)

 Trainable params: 122,832 (479.81 KB)

 Non-trainable params: 2,912 (11.38 KB)

In [67]:
for layer in mobilenetv1liteQuantized.layers[:-1]:
   layer.trainable=True

In [80]:
mobilenetv1liteQuantized.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (32, 256, 256, 8)      │           216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (32, 256, 256, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (32, 256, 256, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (32, 256, 256, 8)      │            72 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (32, 256, 256, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (32, 256, 256, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (32, 256, 256, 16)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (32, 256, 256, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (32, 256, 256, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (32, 127, 127, 16)     │           144 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (32, 127, 127, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (32, 127, 127, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (32, 127, 127, 32)     │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (32, 127, 127, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_2              │ (32, 127, 127, 32)     │           288 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (32, 127, 127, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (32, 127, 127, 32)     │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (32, 127, 127, 32)     │           128 │
│ (BatchNormalization)            │                        │             

 Total params: 125,744 (491.19 KB)

 Trainable params: 122,832 (479.81 KB)

 Non-trainable params: 2,912 (11.38 KB)

In [81]:
mobilenetv1liteQuantized.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [82]:
os.chdir("Air Dataset")

In [44]:
import tensorflow as tf

# Enable eager execution
tf.config.run_functions_eagerly(True)


In [83]:
mobilenetv1liteQuantized.fit(x=train_batches,validation_data=valid_batches,epochs=10,verbose=2)

Epoch 1/10


c:\Users\HP\Documents\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


: 